In [1]:
import requests
from bs4 import BeautifulSoup
import json
import logging
from dataclasses import dataclass
from typing import List, Dict, Optional

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s: %(message)s')

@dataclass
class MenuItem:
    name: str
    original_price: float
    predicted_price: float = None

class APIManager:
    def __init__(self):
        # API Keys (consider using environment variables in production)
        self.yelp_api_key = 'kTVg5Li0g4xuwvedMJsfEfJr06A_Mu5lMeLD2ZXFcixT2X2wP_GifvgRFD1sYalB5_WB5FNN-AET2FIQXWwEGN2XFTgoN5bg1d50oDh5T2AQPXyHiL32tdp8YvZTZ3Yx'
        self.weather_api_key = '10b315d964eb2ddfd41ad9df97cc47d5'

    def get_weather_data(self, lat: float, lon: float) -> Optional[Dict]:
        """Fetch weather data from OpenWeatherMap API"""
        try:
            url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={self.weather_api_key}&units=metric"
            response = requests.get(url)
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            logging.error(f"Weather API Error: {e}")
            return None

    def get_restaurant_menu(self) -> List[MenuItem]:
        """Scrape restaurant menu from Yelp"""
        try:
            url = "https://www.yelp.com/menu/village-the-soul-of-india-hicksville"
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
            }
            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.content, 'html.parser')

            menu_items = soup.find_all('p', class_='menu-item-details-description')
            menu_prices = soup.find_all('li', class_='menu-item-price-amount')

            return [
                MenuItem(
                    name=item.text.strip(), 
                    original_price=float(price.text.strip().replace('$', ''))
                ) 
                for item, price in zip(menu_items, menu_prices)
            ]
        except Exception as e:
            logging.error(f"Menu Scraping Error: {e}")
            return []

class PricePredictionEngine:
    @staticmethod
    def predict_prices(
        menu_items: List[MenuItem], 
        weather_data: Dict, 
        is_busy: bool = True
    ) -> List[MenuItem]:
        """
        Predict menu prices based on weather and demand conditions
        
        Conditions for price increase:
        1. Temperature below 45°F (7.2°C)
        2. Snowing or raining
        3. Restaurant is busy
        """
        # Convert temperature to Fahrenheit
        temp_f = (weather_data['main']['temp'] * 9/5) + 32
        
        # Check weather conditions
        weather_description = weather_data['weather'][0]['description'].lower()
        
        # Determine price adjustment conditions
        is_cold = temp_f < 45
        is_bad_weather = any(condition in weather_description for condition in ['snow', 'rain', 'storm'])
        
        # Price prediction logic
        for item in menu_items:
            if is_cold and is_bad_weather and is_busy:
                # 20% price increase when conditions are met
                item.predicted_price = round(item.original_price * 1.2, 2)
            else:
                # Keep original price
                item.predicted_price = item.original_price
        
        return menu_items

class RestaurantAnalyzer:
    def __init__(self):
        self.api_manager = APIManager()
        self.prediction_engine = PricePredictionEngine()

    def analyze_restaurant_menu(self, lat: float = 40.7128, lon: float = -74.0060):
        """
        Comprehensive restaurant menu and price analysis
        
        Args:
            lat (float): Latitude of location
            lon (float): Longitude of location
        """
        try:
            # Fetch weather data
            weather_data = self.api_manager.get_weather_data(lat, lon)
            if not weather_data:
                logging.error("Could not retrieve weather data")
                return

            # Fetch restaurant menu
            menu_items = self.api_manager.get_restaurant_menu()
            if not menu_items:
                logging.error("Could not retrieve menu items")
                return

            # Predict prices
            predicted_menu = self.prediction_engine.predict_prices(
                menu_items, 
                weather_data, 
                is_busy=True
            )

            # Display results
            self.display_results(predicted_menu, weather_data)

        except Exception as e:
            logging.error(f"Analysis Error: {e}")

    def display_results(self, menu_items: List[MenuItem], weather_data: Dict):
        """Display menu analysis results"""
        print("\n--- Village Restaurant Menu Price Analysis ---")
        print(f"Temperature: {weather_data['main']['temp']}°C")
        print(f"Weather: {weather_data['weather'][0]['description']}")
        
        print("\nMenu Items:")
        for item in menu_items:
            price_change = "Increased" if item.predicted_price > item.original_price else "Unchanged"
            print(f"Item: {item.name}")
            print(f"  Original Price: ${item.original_price:.2f}")
            print(f"  Predicted Price: ${item.predicted_price:.2f}")
            print(f"  Price Change: {price_change}\n")

def main():
    # Create and run restaurant analyzer
    analyzer = RestaurantAnalyzer()
    analyzer.analyze_restaurant_menu()

if __name__ == "__main__":
    main()


--- Village Restaurant Menu Price Analysis ---
Temperature: -2.28°C
Weather: clear sky

Menu Items:
Item: Lentil flour donuts.
  Original Price: $6.00
  Predicted Price: $6.00
  Price Change: Unchanged

Item: Lentil flour donuts smothered with sambar.
  Original Price: $6.99
  Predicted Price: $6.99
  Price Change: Unchanged

Item: Lentil flour donuts smothered with Rasam soup.
  Original Price: $6.99
  Predicted Price: $6.99
  Price Change: Unchanged

Item: Rice cake and lentil donut.
  Original Price: $5.99
  Predicted Price: $5.99
  Price Change: Unchanged

Item: Idly and Vada.
  Original Price: $6.99
  Predicted Price: $6.99
  Price Change: Unchanged

Item: Crispy Chenna Dal mixed with spiced and fried.
  Original Price: $7.45
  Predicted Price: $7.45
  Price Change: Unchanged

Item: Lentil donut soaked in sweet yogurt.
  Original Price: $8.95
  Predicted Price: $8.95
  Price Change: Unchanged

Item: Mashed potato spices and fried with chickpeas flour.
  Original Price: $6.00
  Pr